<a href="https://colab.research.google.com/github/mavroudisv/dpctgan/blob/main/Normalize_SDGym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sdgym

In [ ]:
#Constants
DATASET_NAME = 'census'
HISTOGRAM_BINS = 25
CONFIG_FILENAME = 'normalization_' + DATASET_NAME + '.conf'

In [ ]:
import json
from sdgym.datasets import load_dataset
from sklearn import preprocessing
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#Load data and process metadata
tables = load_dataset(DATASET_NAME)
with open('/root/.local/share/SDGym/datasets/'+ DATASET_NAME + '/metadata.json') as json_file:
    metadata = json.load(json_file)
fields = metadata['tables'][DATASET_NAME]['fields']
numerical_fields = []
for f in fields:
  if fields[f]['type'] != 'categorical':
    numerical_fields.append(f)
numerical_fields
table = tables.load_table(DATASET_NAME)
table

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,race,hispanic origin,sex,member of a labor union,reason for unemployment,full or part time employment stat,capital gains,capital losses,dividends from stocks,tax filer stat,region of previous residence,state of previous residence,detailed household and family stat,detailed household summary in household,migration code-change in msa,migration code-change in reg,migration code-move within reg,live in this house 1 year ago,migration prev res in sunbelt,num persons worked for employer,family members under 18,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,year,label
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Other Rel 18+ ever marr not in subfamily,Other relative of householder,?,?,?,Not in universe under 1 year old,?,0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,- 50000
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Head of household,South,Arkansas,Householder,Householder,MSA to MSA,Same county,Same county,No,Yes,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,- 50000
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Child 18+ never marr Not in a subfamily,Child 18 or older,?,?,?,Not in universe under 1 year old,?,0,Not in universe,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,- 50000
3,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000
4,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299280,14,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,?,?,?,Not in universe under 1 year old,?,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,95,- 50000
299281,61,Private,8,36,11th grade,0,Not in universe,Separated,Manufacturing-durable goods,Machine oper

In [ ]:
#Visualization
for field in numerical_fields:
  table.hist(column=field, bins=HISTOGRAM_BINS)


In [ ]:
# Generate Config file
config = {}
for field in numerical_fields:
  config[field] = [table[field].min(),table[field].max()]
  

with open(CONFIG_FILENAME, 'w') as fp:
    json.dump(config, fp)



In [ ]:
# Clip and normalize
with open(CONFIG_FILENAME) as json_file:
    config = json.load(json_file)
config


#Clip
for field in config:
  lower_value = config[field][0]
  upper_value = config[field][1]
  table[field] = table[field].clip(lower=lower_value, upper=upper_value)


#Normalize
table_normalized = table.copy()
x = table[config.keys()].values
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=config.keys(), index = table.index)
table_normalized[list(config.keys())] = df_temp
table_normalized

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,race,hispanic origin,sex,member of a labor union,reason for unemployment,full or part time employment stat,capital gains,capital losses,dividends from stocks,tax filer stat,region of previous residence,state of previous residence,detailed household and family stat,detailed household summary in household,migration code-change in msa,migration code-change in reg,migration code-move within reg,live in this house 1 year ago,migration prev res in sunbelt,num persons worked for employer,family members under 18,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,year,label
0,0.811111,Not in universe,0.000000,0.000000,High school graduate,0.0,Not in universe,Widowed,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Other Rel 18+ ever marr not in subfamily,Other relative of householder,?,?,?,Not in universe under 1 year old,?,0.000000,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,1.0,0.0,1.0,- 50000
1,0.644444,Self-employed-not incorporated,0.078431,0.739130,Some college but no degree,0.0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Head of household,South,Arkansas,Householder,Householder,MSA to MSA,Same county,Same county,No,Yes,0.166667,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,1.0,1.0,0.0,- 50000
2,0.200000,Not in universe,0.000000,0.000000,10th grade,0.0,High school,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,All other,Female,Not in universe,Not in universe,Not in labor force,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child 18+ never marr Not in a subfamily,Child 18 or older,?,?,?,Not in universe under 1 year old,?,0.000000,Not in universe,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0.0,Not in universe,1.0,0.0,1.0,- 50000
3,0.100000,Not in universe,0.000000,0.000000,Children,0.0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0.000000,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,0.0,0.0,0.0,- 50000
4,0.111111,Not in universe,0.000000,0.000000,Children,0.0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0.000000,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,0.0,0.0,0.0,- 50000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299280,0.155556,Not in universe,0.000000,0.000000,Children,0.0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,?,?,?,Not in universe under 1 year old

In [ ]:
np.savez(DATASET_NAME+"_norm", table_normalized.values)

In [ ]:
#Clip+normalization packed into a function

import json
from sdgym.datasets import load_dataset
from sklearn import preprocessing
import pandas as pd
import numpy as np


# Generate the default config file
def gen_config(dataset_name):
  config_filename = 'normalization_' + dataset_name + '.conf'

  #Load dataset
  tables = load_dataset(DATASET_NAME)
  with open('/root/.local/share/SDGym/datasets/'+ DATASET_NAME + '/metadata.json') as json_file:
      metadata = json.load(json_file)
  fields = metadata['tables'][DATASET_NAME]['fields']
  numerical_fields = []
  for f in fields:
    if fields[f]['type'] != 'categorical':
      numerical_fields.append(f)
  table = tables.load_table(DATASET_NAME)


  # Generate Config file
  config = {}
  for field in numerical_fields:
    config[field] = [table[field].min(),table[field].max()]
    
  with open(config_filename, 'w') as fp:
      json.dump(config, fp)


######################################################################

#Load and normalize dataset
def load_dataset_norm(dataset_name):
  config_filename = 'normalization_' + dataset_name + '.conf'

  #Load dataset
  tables = load_dataset(DATASET_NAME)
  with open('/root/.local/share/SDGym/datasets/'+ DATASET_NAME + '/metadata.json') as json_file:
      metadata = json.load(json_file)
  fields = metadata['tables'][DATASET_NAME]['fields']
  numerical_fields = []
  for f in fields:
    if fields[f]['type'] != 'categorical':
      numerical_fields.append(f)
  table = tables.load_table(DATASET_NAME)

  #Load config
  try:
    with open(config_filename) as json_file:
        config = json.load(json_file)
  except:
    print('There is something wrong with the normalization config file. Does it even exist?')  


  #Clip
  for field in config:
    lower_value = config[field][0]
    upper_value = config[field][1]
    table[field] = table[field].clip(lower=lower_value, upper=upper_value)


  #Normalize
  table_normalized = table.copy()
  x = table[config.keys()].values
  min_max_scaler = MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  df_temp = pd.DataFrame(x_scaled, columns=config.keys(), index = table.index)
  table_normalized[list(config.keys())] = df_temp
  return table_normalized


######################################################################

#Usage examples
gen_config('census') #Generate default config file
load_dataset_norm('census') #Normalize


,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,race,hispanic origin,sex,member of a labor union,reason for unemployment,full or part time employment stat,capital gains,capital losses,dividends from stocks,tax filer stat,region of previous residence,state of previous residence,detailed household and family stat,detailed household summary in household,migration code-change in msa,migration code-change in reg,migration code-move within reg,live in this house 1 year ago,migration prev res in sunbelt,num persons worked for employer,family members under 18,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,year,label
0,0.811111,Not in universe,0.000000,0.000000,High school graduate,0.0,Not in universe,Widowed,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Other Rel 18+ ever marr not in subfamily,Other relative of householder,?,?,?,Not in universe under 1 year old,?,0.000000,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,1.0,0.0,1.0,- 50000
1,0.644444,Self-employed-not incorporated,0.078431,0.739130,Some college but no degree,0.0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Head of household,South,Arkansas,Householder,Householder,MSA to MSA,Same county,Same county,No,Yes,0.166667,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,1.0,1.0,0.0,- 50000
2,0.200000,Not in universe,0.000000,0.000000,10th grade,0.0,High school,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,All other,Female,Not in universe,Not in universe,Not in labor force,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child 18+ never marr Not in a subfamily,Child 18 or older,?,?,?,Not in universe under 1 year old,?,0.000000,Not in universe,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0.0,Not in universe,1.0,0.0,1.0,- 50000
3,0.100000,Not in universe,0.000000,0.000000,Children,0.0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0.000000,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,0.0,0.0,0.0,- 50000
4,0.111111,Not in universe,0.000000,0.000000,Children,0.0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0.000000,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,0.0,0.0,0.0,- 50000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299280,0.155556,Not in universe,0.000000,0.000000,Children,0.0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,?,?,?,Not in universe under 1 year old